<a href="https://colab.research.google.com/github/hood-boi/world-news-chatbot/blob/master/World_News_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Extraction

In [0]:
import requests
import json
import time

In [0]:
!pip install langdetect

     |████████████████████████████████| 1.0MB 6.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/ec/0c/a9/1647275e7ef5014e7b83ff30105180e332867d65e7617ddafe
Successfully built langdetect


In [0]:
from langdetect import detect

In [0]:
detect("Hello my name is Himanshi")

'en'

In [0]:
# -*- coding: utf-8 -*-
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

In [0]:
def updateListingDict(postsJSON, postsDICT):
    chunk_size = len(postsJSON["data"]["children"])
    posts = postsJSON["data"]["children"]
    
    for i in range(chunk_size): #Loop through each reddit post
        r_post_title = posts[i]["data"]["title"];
        r_post_title = " ".join(r_post_title.split());
        r_post_title = r_post_title.replace('\n', ' ');
        r_post_title = r_post_title.replace('\t', ' ');
        r_post_title = r_post_title.replace('\'', '');
        r_post_title = r_post_title.replace('\"', '');

        postsDICT[posts[i]["data"]["id"]] = r_post_title;

        
    

after = '';
listing_dict = {};

for i in range(0, 10):
    if(after):
        print("[AFTER] = ", after)
        r_worldnews = requests.get('https://www.reddit.com/r/worldnews/top.json?t=all&limit=100&after='+after, 
                                headers = { 'User-agent' : 'ChatBot', 
                                            'Accept' : '*/*',
                                            'Cache-Control' : 'no-cache'
                                          });
        if(r_worldnews.status_code == 200):
            postsJSON = r_worldnews.json()
            updateListingDict(postsJSON, listing_dict);
            after = postsJSON["data"]["after"];
    else:
        print("[AFTER] = NONE")
        r_worldnews = requests.get('https://www.reddit.com/r/worldnews/top.json?t=all&limit=100', 
                                headers = { 'User-agent' : 'ChatBot', 
                                            'Accept' : '*/*',
                                            'Cache-Control' : 'no-cache' 
                                          });
        if(r_worldnews.status_code == 200):
            postsJSON = r_worldnews.json()
            updateListingDict(postsJSON, listing_dict);
            #print("[GET] request successful")
            after = postsJSON["data"]["after"];
            
            


In [0]:
print(len(listing_dict))

for key, value in listing_dict.items():
    print(key, value)
    break;

82
901p5f Two weeks before his inauguration, Donald J. Trump was shown highly classified intelligence indicating that President Vladimir V. Putin of Russia had personally ordered complex cyberattacks to sway the 2016 American election


In [0]:
print(key,value)   

901p5f Two weeks before his inauguration, Donald J. Trump was shown highly classified intelligence indicating that President Vladimir V. Putin of Russia had personally ordered complex cyberattacks to sway the 2016 American election


In [8]:
from google.colab import drive
drive.mount('/content/gdrive')
#4/igGrpBUzgmmOTdhG77m8_Ej4eL8V8wcgV1kMDSd2zZel0dhlp_w-eek

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
! mkdir /content/gdrive/My\ Drive/Chatbot

mkdir: cannot create directory ‘/content/gdrive/My Drive/Chatbot’: File exists


In [0]:
! rm /content/gdrive/My\ Drive/Chatbot/rWorldNews.txt

rm: cannot remove '/content/gdrive/My Drive/Chatbot/rWorldNews.txt': No such file or directory


In [0]:
file_handle = open('/content/gdrive/My Drive/Chatbot/rWorldNews.txt', 'a+')

#Expecting array of JSON children
def recursePostComments(context, commentsJSON):
   
    size = len(commentsJSON)
    for i in range(0, size):
        #print(commentsJSON[i])
        if('data' in commentsJSON[i].keys()):
            if('body' in commentsJSON[i]['data'].keys()):
                reply = commentsJSON[i]['data']['body'];
                if("[deleted]" in reply or "[removed]" in reply):
                    continue;
                else:
                    reply = reply.replace('\n', ' ');
                    reply = reply.replace('\t', ' ');
                    reply = reply.replace('\'', '');
                    reply = reply.replace('\"', '');
                    reply = reply.replace('&gt', '');
                    reply = reply.replace('&lt', '');
                    #print("Context : ", context);
                    
                    #if (len(reply) > 500):
                        #print("\tLen :", len(reply));
                    #    t_reply = summarizeText(reply);
                    #    if (len(t_reply) > 0):
                    #        reply = t_reply;
                    #print("\tReply :", reply);
                    
                    context.replace('\t', ' ');
                    
                    reply.replace('\t', ' ');
                    reply = " ".join(reply.split());
                    
                    if("Article has nothing" in context):
                        print("NIqqa")
                    
                    if(reply.isspace() or context.isspace()):
                        print("Error : ", reply, ", ", context)
                        continue;                    
                    else:
                        line_to_write = "{}\t{}\n".format(reply.strip(), context.strip());
                        file_handle.write(line_to_write)
                    
                    if('replies' in commentsJSON[i]['data'].keys()):
                        if(commentsJSON[i]['data']['replies']):
                            if('data' in commentsJSON[i]['data']['replies'].keys()):
                                if(commentsJSON[i]['data']['replies']['data']):
                                    if('children' in commentsJSON[i]['data']['replies']['data'].keys()):
                                        recursePostComments(reply, commentsJSON[i]['data']['replies']['data']['children']);
            else:
                continue;
        else:
            continue;
    return;



In [0]:
for key, value in listing_dict.items():
    print(key, value)
    r_comments = requests.get('https://www.reddit.com/r/worldnews/comments/' + key +'.json?limit=10000&sort=confidence&depth=4', 
                                headers = { 'User-agent' : 'ChatBot', 
                                            'Accept' : '*/*',
                                            'Cache-Control' : 'no-cache' 
                                          });
    r_comments_json = r_comments.json();
    context = value;
    commentsJSON = r_comments_json[1]["data"]["children"]
    recursePostComments(context, commentsJSON)
    
file_handle.close()

#Overfitting and Testing Generative model

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
import torchtext
import random


In [0]:
text_field = torchtext.data.Field(sequential=True,      # text sequence
                                  tokenize=lambda x: x, # because are building a character-RNN
                                  include_lengths=True, # to track the length of sequences, for batching
                                  batch_first=True,
                                  use_vocab=True,
                                  init_token="<BOS>",
                                  eos_token="<EOS>"
                                 )       # to turn each character into an integer index
label_field = torchtext.data.Field(sequential=True,    # text sequence
                                   use_vocab=True,     # don't need to track vocabulary
                                   is_target=True,      
                                   batch_first=True,
                                   tokenize=lambda x: x,
                                   preprocessing=lambda x: x,
                                   init_token="<BOS>",
                                   eos_token="<EOS>"
                                  ) 

fields = [('reply', label_field), ('context', text_field)]
dataset = torchtext.data.TabularDataset("/content/gdrive/My Drive/Chatbot/rWorldNews.txt", # name of the file
#dataset = torchtext.data.TabularDataset("test.txt",
                                        "tsv",               # fields are separated by a tab
                                        fields)

In [4]:
len(dataset)

424044

In [0]:
train = torchtext.data.Dataset(dataset, fields)

In [6]:
len(train)

424044

In [0]:
text_field.build_vocab(train)
label_field.build_vocab(train)

In [8]:
print(text_field.vocab.itos)
print(label_field.vocab.itos)

['<unk>', '<pad>', '<BOS>', '<EOS>', ' ', 'e', 't', 'a', 'o', 'i', 'n', 's', 'r', 'h', 'l', 'd', 'u', 'c', 'm', 'p', 'g', 'y', 'w', 'f', 'b', '.', 'v', 'k', ',', 'I', '-', '/', 'T', 'S', 'A', '0', '1', 'C', 'x', '*', ':', '2', 'j', 'P', 'M', 'W', 'R', 'E', 'N', '?', 'B', 'H', 'U', ')', 'D', '(', 'O', 'F', '’', 'z', ';', '5', '3', 'G', '_', '8', '9', '6', 'L', '4', '7', 'K', 'q', ']', '[', 'Y', 'J', '!', 'V', '%', '“', '”', '=', '$', '&', '#', '~', '^', '|', 'Q', 'Z', '‘', '—', 'X', '+', '\\', '–', '@', 'é', '£', '…', 'о', 'ü', 'е', '€', '•', '⠀', 'ó', 'и', 'н', '️', 'а', 'т', 'á', 'ñ', 'с', '°', '・', 'ä', 'л', 'р', 'ا', '§', '\u200b', 'м', 'ы', 'в', '`', 'к', 'ن', 'д', '´', '✔', '⣿', 'п', 'í', 'ی', 'ö', '，', 'я', '😂', 'б', 'у', '。', '🤔', '\u200d', '💦', 'ر', '¥', 'م', 'ه', 'の', 'が', 'ч', 'º', '🇧', 'و', '¯', '·', '■', 'Ä', '‚', 'ل', '¶', 'た', 'な', '🇬', 'د', 'ت', '👍', 'г', '不', 'か', 'て', 'ب', 'っ', '人', '★', '👏', '、', 'い', '☆', 'ğ', 'Ü', 'ь', '™', '一', 'з', '💘', '₂', '交', '全', '！', '🇺', 'й

In [0]:
train_iter = torchtext.data.BucketIterator(train,
                                           batch_size=1,
                                           sort_key=lambda x: len(x.context), # to minimize padding
                                           sort_within_batch=True,        # sort within each batch
                                           repeat=False,
                                          )                  # repeat the iterator for many epochs

In [17]:
len(train_iter)

424044

In [18]:
k = 0;
for batch in train_iter:
    
    #max_length_seq = batch.context[0].shape[1];
    #print("-----Batch #", k+1, "-----");
    #print("Maximum length of the input sequence : ", max_length_seq);
    #padding_per_batch = int(torch.sum(max_length_seq - batch.sms[1]))
    #print("Number of padding for batch : ", padding_per_batch)
    
    print("Batch Size : ", len(batch))
    #print("Batch Context : ", batch.context)
    print("Batch Context Shape: ", batch.context[0].shape)
    #print(batch.reply)
    print("Batch Context Shape: ", batch.reply.shape)
    k += 1;
    
    if(k >= 1):
        break;

Batch Size :  1
Batch Context Shape:  torch.Size([1, 1067])
Batch Context Shape:  torch.Size([1, 11])


In [0]:
def embedding_to_string(token, context_field, reply_field):
    context_array =[];
    for i in range(0, token.context[0].shape[1]):
        context_array.append(context_field.vocab.itos[(token.context[0][0][i])])

    context_str = ''.join(context_array)



    reply_array =[];
    for i in range(0, token.reply.shape[1]):
        reply_array.append(reply_field.vocab.itos[(token.reply[0][i])])

    reply_str = ''.join(reply_array)

    print("Context : ", context_str);
    print("Reply : ", reply_str);

In [20]:
embedding_to_string(batch, text_field, label_field)

Context :  <BOS>providence mistype repudiation postdates offenders bankrolls echoed Tutankhamen keynotes furloughed unacceptably entrenched diced cradle nutritionists doodlers abroad Vonda soliloquized garnisheed murderesss biennial reawakening zenith crappy Tartary Goren appendicitis scrods style basilica boozy captivated gored dubietys mannishly drill midwife machinations inventiveness bellyaches Marquez bys fetch joggling emaciating hurtled varnished awarded keypunch chicory taught showers freeloaders cervix dailies rearwards descendant Duran boastfulnesss bowsprits Bulfinch timekeepers shaved imposed underwear ejected putts Antichrists stalkings Shinto superegos newsboy Blanche eights bludgeon perfections watermark nab pigpen slipping cosmoses sunbathings chauvinist reign cubicles watched paunchiest youngish subtotals minuscule stripper stateroom campsites decapitates monkeys tallnesss cosmopolitans welled easterlies Snell enslave gazettes ogled peripherys Pickwicks straggler recor

In [13]:
input_vocab_size = len(text_field.vocab.itos)
reply_vocab_size = len(label_field.vocab.itos)
print("Input Vocab Size: ", input_vocab_size)
print("Reply Vocab size: ", reply_vocab_size)

Input Vocab Size:  1277
Reply Vocab size:  2232


In [22]:
print(batch.context[0][0])
print(batch.reply)

tensor([ 2, 19, 12,  ..., 12, 11,  3])
tensor([[ 2, 59, 12,  8,  6, 13,  9, 10, 20, 24,  3]])


In [0]:
criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(model.parameters(), lr = 0.01);

In [24]:
#>>> BEGIN: Encoding Region

context_ident = torch.eye(input_vocab_size)
print("Shape 1 : ", batch.context[0].shape) # (batch size, sequence size)
context_tensor = context_ident[batch.context[0]]

print("Shape 2 : ", context_tensor.shape) # (batch size, sequence size, one hot embedding per size)
encode_rnn = nn.GRU(input_vocab_size, 100, 1, batch_first=True)
h0 = torch.zeros(1, batch.context[0].shape[0], 100); # (num layers * direction, batch size, hidden size)
out, last_hidden = encode_rnn(context_tensor, h0)
print("Shape 3 : ", out.shape)
print("Shape 4 : ", last_hidden.shape)

#<<< END: Encoding Region

#>>> BEGIN: Teach forcing Generation Region
reply_ident = torch.eye(reply_vocab_size)
print("Shape 5 : ", batch.reply.shape)
reply_tensor = reply_ident[batch.reply]

print("Shape 6 : ", reply_tensor.shape)
decode_rnn = nn.GRU(reply_vocab_size, 100, 1, batch_first=True)
out2, last_hidden2 = decode_rnn(reply_tensor[:,:-1,:], last_hidden) # Don't pass in <EOS> token
print("Shape 7 : ", out2.shape)
print("Shape 8 : ", last_hidden2.shape)

target = reply_tensor[:,1:,:]
print("Shape 9 : ", target.shape)

fcnn = nn.Linear(100, reply_vocab_size)

final_destination = fcnn(out2)

print("Shape 10 : ", final_destination.shape)


temp1 = final_destination.reshape(-1, reply_vocab_size);
temp2 = batch.reply[:,1:].reshape(-1)

print("Shape 11 : ", temp1.shape)
print("Shape 12 : ", temp2.shape)

loss = criterion(temp1, temp2)
print("Loss : ", loss.item())

#<<< END: Teach forcing Generation Region





Shape 1 :  torch.Size([1, 1067])
Shape 2 :  torch.Size([1, 1067, 1277])
Shape 3 :  torch.Size([1, 1067, 100])
Shape 4 :  torch.Size([1, 1, 100])
Shape 5 :  torch.Size([1, 11])
Shape 6 :  torch.Size([1, 11, 2232])
Shape 7 :  torch.Size([1, 10, 100])
Shape 8 :  torch.Size([1, 1, 100])
Shape 9 :  torch.Size([1, 10, 2232])
Shape 10 :  torch.Size([1, 10, 2232])
Shape 11 :  torch.Size([10, 2232])
Shape 12 :  torch.Size([10])
Loss :  7.7212066650390625


In [0]:
class ChatBot(nn.Module):
    def __init__(self, 
                 context_vocab_size,  
                 reply_vocab_size, 
                 encoder_hidden_size = 100,
                 generator_hidden_size = 100, 
                 encoder_layers = 1, 
                 generator_layers = 1):
        
        super(ChatBot, self).__init__()
        
        self.encoder_layers = encoder_layers;
        self.generator_layers =generator_layers;
        self.encoder_hidden_size = encoder_hidden_size;
        self.generator_hidden_size = generator_hidden_size;
        
        # >>> Encoder
        self.context_ident = torch.eye(context_vocab_size)
        self.encode_rnn = nn.LSTM(context_vocab_size, encoder_hidden_size, encoder_layers, batch_first=True)
        
        # >>> Generator
        self.reply_ident = torch.eye(reply_vocab_size)
        self.decode_rnn = nn.LSTM(reply_vocab_size, generator_hidden_size, generator_layers, batch_first=True)
        self.fcnn = nn.Linear(generator_hidden_size, reply_vocab_size)
        
    def forward(self, context, response, hidden=None):
        
        # >>> Encoder
        context_tensor = self.context_ident[context] # Type: batch.context[0] | Size: (batch size, sequence size)
        h0 = torch.zeros(self.encoder_layers, context.shape[0], self.encoder_hidden_size); # (num layers * direction, batch size, hidden size)
        c0 = torch.zeros(self.encoder_layers, context.shape[0], self.encoder_hidden_size);
        encode_out, encode_last_hidden = self.encode_rnn(context_tensor, (h0,c0))
        
        # >>> Generator
        reply_tensor = self.reply_ident[response] #Type: batch.reply
        if(hidden == None):
            gen_out, gen_last_hidden = self.decode_rnn(reply_tensor, encode_last_hidden)
        else:
            gen_out, gen_last_hidden = self.decode_rnn(reply_tensor, hidden)
        out = self.fcnn(gen_out)
        return out, gen_last_hidden

In [0]:
model = ChatBot(input_vocab_size, reply_vocab_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01);

In [27]:
for i in range(0,100):
    optimizer.zero_grad();
    out, hidden = model(batch.context[0], batch.reply[:, :-1]) # no eos
    #print(batch.reply[:, :-1])
    out_reshaped = out.reshape(-1,reply_vocab_size)
    target = (batch.reply[:,1:]).reshape(-1) # no bos
    #print(batch.reply[:,1:])
    loss = criterion(out_reshaped, target)
    print(loss)
    loss.backward();
    optimizer.step();

tensor(7.7210, grad_fn=<NllLossBackward>)
tensor(7.5697, grad_fn=<NllLossBackward>)
tensor(7.2976, grad_fn=<NllLossBackward>)
tensor(6.0054, grad_fn=<NllLossBackward>)
tensor(4.0020, grad_fn=<NllLossBackward>)
tensor(2.6061, grad_fn=<NllLossBackward>)
tensor(2.0337, grad_fn=<NllLossBackward>)
tensor(1.8325, grad_fn=<NllLossBackward>)
tensor(1.6938, grad_fn=<NllLossBackward>)
tensor(1.6092, grad_fn=<NllLossBackward>)
tensor(1.5348, grad_fn=<NllLossBackward>)
tensor(1.4921, grad_fn=<NllLossBackward>)
tensor(1.3814, grad_fn=<NllLossBackward>)
tensor(1.3395, grad_fn=<NllLossBackward>)
tensor(1.2764, grad_fn=<NllLossBackward>)
tensor(1.2310, grad_fn=<NllLossBackward>)
tensor(1.1848, grad_fn=<NllLossBackward>)
tensor(1.1503, grad_fn=<NllLossBackward>)
tensor(1.0925, grad_fn=<NllLossBackward>)
tensor(1.0589, grad_fn=<NllLossBackward>)
tensor(1.0387, grad_fn=<NllLossBackward>)
tensor(0.9969, grad_fn=<NllLossBackward>)
tensor(0.9572, grad_fn=<NllLossBackward>)
tensor(0.9210, grad_fn=<NllLossBac

In [0]:
label_field.vocab.stoi["<BOS>"]

2

In [0]:
def sample_sequence(model, max_len=100, temperature=0.8):
    generated_sequence = ""
   
    inp = torch.Tensor([label_field.vocab.stoi["<BOS>"]]).long()
    hidden = None;
    for p in range(max_len):
        #print(inp)
        output, hidden = model(batch.context[0], inp.unsqueeze(0), hidden)
        #print(output)
        #output = F.softmax(output, dim=2)
        #print(torch.argmax(output, dim=2))
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        #print(output_dist)
        top_i = int(torch.multinomial(output_dist, 1)[0])
        # Add predicted character to string and use as next input
        predicted_char = label_field.vocab.itos[top_i]
        
        if predicted_char == "<EOS>":
            break
        generated_sequence += predicted_char       
        inp = torch.Tensor([top_i]).long()
    return generated_sequence




In [29]:
sample_sequence(model, temperature=1.0)

'Frothinn.'

In [0]:
inp = torch.Tensor([label_field.vocab.stoi["<BOS>"]]).long()
print(inp.shape)

#Baseline Model

In [0]:
class ContextAE(nn.Module):
    def __init__(self, 
                 context_vocab_size, 
                 encoder_hidden_size = 100,
                 generator_hidden_size = 100, 
                 encoder_layers = 1, 
                 generator_layers = 1):
        
        super(ContextAE, self).__init__()
        
        self.encoder_layers = encoder_layers;
        self.generator_layers = generator_layers;
        self.encoder_hidden_size = encoder_hidden_size;
        self.generator_hidden_size = generator_hidden_size;
        
        # >>> Encoder
        self.context_ident = torch.eye(context_vocab_size)
        self.encode_rnn = nn.LSTM(context_vocab_size, encoder_hidden_size, encoder_layers, batch_first=True)
        
        # >>> Decoder
        self.reply_ident = torch.eye(context_vocab_size)
        self.decode_rnn = nn.LSTM(context_vocab_size, generator_hidden_size, generator_layers, batch_first=True)
        self.fcnn = nn.Linear(generator_hidden_size, context_vocab_size)
        
    def forward(self, context, hidden=None):
        
        # >>> Encoder
        context_tensor = self.context_ident[context] # Type: batch.context[0] | Size: (batch size, sequence size)
        h0 = torch.zeros(self.encoder_layers, context.shape[0], self.encoder_hidden_size); # (num layers * direction, batch size, hidden size)
        c0 = torch.zeros(self.encoder_layers, context.shape[0], self.encoder_hidden_size);
        encode_out, encode_last_hidden = self.encode_rnn(context_tensor, (h0,c0))
        
        # >>> Decoder
        if(hidden == None):
            gen_out, gen_last_hidden = self.decode_rnn(context_tensor, encode_last_hidden)
        else:
            gen_out, gen_last_hidden = self.decode_rnn(context_tensor, hidden)
        out = self.fcnn(gen_out)
        return out, gen_last_hidden

In [30]:
torch.cuda.is_available()

True

In [0]:
train_iter = torchtext.data.BucketIterator(train,
                                           batch_size=16,
                                           sort_key=lambda x: len(x.context), # to minimize padding
                                           sort_within_batch=True,        # sort within each batch
                                           repeat=False,
                                          )                  # repeat the iterator for many epochs

In [0]:
model_AE = ContextAE(input_vocab_size, encoder_hidden_size = 200, generator_hidden_size = 200 )
criterion = nn.CrossEntropyLoss()
optimizer_AE = torch.optim.Adam(model_AE.parameters(), lr = 0.01);

In [50]:
for batch in train_iter:
    
    #max_length_seq = batch.context[0].shape[1];
    #print("-----Batch #", k+1, "-----");
    #print("Maximum length of the input sequence : ", max_length_seq);
    #padding_per_batch = int(torch.sum(max_length_seq - batch.sms[1]))
    #print("Number of padding for batch : ", padding_per_batch)
    
    print("Batch Size : ", len(batch))
    #print("Batch Context : ", batch.context)
    print("Batch Context Shape: ", batch.context[0].shape)
    #print(batch.reply)
    print("Batch Context Shape: ", batch.reply.shape)
    
    to_encode = batch.context[0];
    
    optimizer_AE.zero_grad();
    out, hidden = model_AE(to_encode[:,1:]) # no eos
    #print(batch.reply[:, :-1])
    out_reshaped = out.reshape(-1,input_vocab_size)
    target = (to_encode[:,1:]).reshape(-1) # no bos
    #print(batch.reply[:,1:])
    loss = criterion(out_reshaped, target)
    print(loss)
    loss.backward();
    optimizer_AE.step();

Batch Size :  16
Batch Context Shape:  torch.Size([16, 123])
Batch Context Shape:  torch.Size([16, 832])
tensor(7.1478, grad_fn=<NllLossBackward>)
Batch Size :  16
Batch Context Shape:  torch.Size([16, 144])
Batch Context Shape:  torch.Size([16, 1716])
tensor(6.9274, grad_fn=<NllLossBackward>)
Batch Size :  16
Batch Context Shape:  torch.Size([16, 635])
Batch Context Shape:  torch.Size([16, 1097])
tensor(4.3070, grad_fn=<NllLossBackward>)
Batch Size :  16
Batch Context Shape:  torch.Size([16, 9849])
Batch Context Shape:  torch.Size([16, 1472])
tensor(3.2265, grad_fn=<NllLossBackward>)
Batch Size :  16
Batch Context Shape:  torch.Size([16, 434])
Batch Context Shape:  torch.Size([16, 1310])
tensor(3.2517, grad_fn=<NllLossBackward>)
Batch Size :  16
Batch Context Shape:  torch.Size([16, 201])
Batch Context Shape:  torch.Size([16, 352])
tensor(3.4143, grad_fn=<NllLossBackward>)
Batch Size :  16
Batch Context Shape:  torch.Size([16, 81])
Batch Context Shape:  torch.Size([16, 457])
tensor(3.

KeyboardInterrupt: ignored